# Push your Model to the Hub

> Have a trained model that you want to push to the Hub? Here's your ticket.

**Prerequisites: you need to sign up for a Huggingface account [on their site](https://huggingface.co/)**

This notebook covers how to use the `hf_hub_push` function to push your model to the Hub. This notebook is based on the documentation provided by HuggingFace in their course, specifically [this chapter](https://huggingface.co/course/chapter4/3?fw=pt).

Documentation on the **Huggingface Hub** can be found [here](https://huggingface.co/docs/hub/main).

## getting started

> Note: **before doing anything further on this notebook**, you need to have a HuggingFace account and a model saved to your local machine. Download generated models saved to your google drive from the training notebook, or you can practice with demo models downloaded with the `download_models.py` script in the root of this repository.



In [1]:
from huggingface_hub import (
    # User management
    login,
    logout,
    notebook_login,
    whoami,
    # Repository creation and management
    create_repo,
    delete_repo,
    update_repo_visibility,
    # And some methods to retrieve/change information about the content
    list_models,
    list_datasets,
    list_metrics,
    list_repo_files,
    upload_file,
    delete_file,
)

login to your huggingface account below.

In [2]:
notebook_login()

the below confirms whether you are logged in to the Hub, and will also display your organization and account information.

(you get the org API token from here for use below)

In [3]:
from IPython.display import clear_output
import time
import pprint as pp

pp.pprint(whoami())  # print the user's name and other info
time.sleep(20)  # clear_output after 20 seconds
clear_output(wait=False)

---

## Setup your repo

enter the repo name in the generated widget below. this will be used as the repository _and the model name_ in the Hub.

In [4]:
from pathlib import Path
from huggingface_hub import list_repo_files, Repository
from ipywidgets import widgets

my_repo = widgets.Text(
    value="My Hello World Repo",
    placeholder="put repo name here",
    description="Enter the name of the repository: ",
    disabled=False,
)


my_repo

Text(value='My Hello World Repo', description='Enter the name of the repository: ', placeholder='put repo name…

In [5]:
print(f"your repository {my_repo.value} will be created")

your repository testing-aimsgbot-upload will be created


Enter the filepath of the folder of the `aitextgen` model to upload. aitextgen's
Because we are using aitextgen, this is the folder containing the model's `config.json` and `pytorch_model.bin` files.

_Note: if you already have an aitextgen model fully exported, that's fine too_

In [6]:
local_path = widgets.Text(
    placeholder="Enter the path of the folder to upload",
    description="Filepath: ",
    disabled=False,
)
local_path

Text(value='', description='Filepath: ', placeholder='Enter the path of the folder to upload')

In [7]:
print(f"your local path \n {local_path.value} \n will be uploaded")

your local path 
 C:\Users\peter\code-dev-22\chatbot-conversation\GPT2_trivNatQAdailydia_774M_175Ksteps 
 will be uploaded


Enter the org API token if you wish to upload to an organization profile vs. your profile. If you do not enter an API token, you will be uploading to your profile.

- it is worth noting that the owner of the model can be changed later (in both directions) via web interface.

In [8]:
org_token = widgets.Password(
    placeholder="Enter your org token", description="Org Token:", disabled=False
)
print("enter your org token below to publish to your org. otherwise, leave blank.\n")
org_token

enter your org token below to publish to your org. otherwise, leave blank.



Password(description='Org Token:', placeholder='Enter your org token')

the below cell will confirm whether you were successful in adding your org API token to the notebook.

In [9]:
use_org = True if org_token.value else False
org_box = widgets.Text(
    placeholder="Enter your org name",
    description="Name of org on hf.co: ",
    disabled=False,
)
print("If relevant, enter your org name below.\n")
print(f"Have a stored org token with length: {len(org_token.value)} chars.")
conf_word = "will" if use_org else "will not"
print(f"therefore this notebook {conf_word} be published to your org.")
org_box if use_org else ""

If relevant, enter your org name below.

Have a stored org token with length: 42 chars.
therefore this notebook will be published to your org.


Text(value='', description='Name of org on hf.co: ', placeholder='Enter your org name')

---

## create repo on hub, clone to local

This section basically uses the `huggingface_hub` function to create a repository on the online Hub, and then clones that repository to your local machine.
No action needs to be completed here outside of running the cells, but you will need to confirm that the repo was created and cloned successfully.

In [12]:
nb_dir = (
    Path().cwd()
)  # note that if the notebook is moved, this will not work and you will need to update this
root = nb_dir.parent

repo_name = str(my_repo.value)
repo_path = root / repo_name  # clone to local working directory
repo_url = (
    create_repo(name=repo_name, exist_ok=True)
    if not use_org
    else create_repo(
        repo_type=None,
        name=repo_name,
        organization=org_box.value,
        exist_ok=True,
    )
)

repo = Repository(
    local_dir=str(repo_path.resolve()),
    clone_from=repo_url,
    use_auth_token=True,
)

Cloning https://huggingface.co/ethzanalytics/testing-aimsgbot-upload into local empty directory.


## export aitextgen model to repo folder

- based on the directory specified previously, run the `prep_huggingface.py` script to convert the (_likely aitextgen_) model to the format required by the Hub
- Ultimately, it will save the relevant model files to the location created in the cell above, for example: `https://huggingface.co/pszemraj/Ballpark-Trivia-M`

In [13]:
import subprocess

# if new to subprocess, see https://docs.python.org/3/library/subprocess.html - basically just a wrapper around os.system to run shell commands
# run deploy-as-bot\prep_huggingface.py with the repo name as an argument
subprocess.run(
    [
        "python",
        "deploy-as-bot/prep_huggingface.py",
        "--model-dir",
        local_path.value,
        "--hf-name",
        repo_name,
    ],
    cwd=root.resolve(),
)

CompletedProcess(args=['python', 'deploy-as-bot/prep_huggingface.py', '--model-dir', 'C:\\Users\\peter\\code-dev-22\\chatbot-conversation\\GPT2_trivNatQAdailydia_774M_175Ksteps', '--hf-name', 'testing-aimsgbot-upload'], returncode=0)

## push to hub - upload like it's hot

In [14]:
commit_msg = widgets.Text(
    placeholder="DEINE GIT KOMMIT", description="Commit message: ", disabled=False
)
commit_msg  # enter the commit message here

Text(value='', description='Commit message: ', placeholder='DEINE GIT KOMMIT')

In [ ]:
repo.push_to_hub(str(commit_msg.value))

---